# [写在前面] 背景

现在云端很容易获得高效廉价的算力和资源，但是通常只能用在特定工作模式下，很不方便，包括但不限于:
1. [google] Colab
2. [google] IDX
3. many more?

# [写在前面] 未来事宜:

1. 调通: rustdesk seemless 模式启动 x-server，目测效果有限，因为手动需要定制的很多，自动模式功能有限?
2. 修复: 声音支持，原来可以，后来不知道哪里改的又不行了？
3. 优化X启动: 目测调用/etc/X11/Xsession是一个更好的做法？避免自己管理一堆上下文？
   /etc/X11/Xsession会自动处理dbus等一系列问题？ 通过dbus-x11增加进入/etc/X11/Xsession.d的文件可知
4. rustdesk调试方案-log: https://rust-lang-nursery.github.io/rust-cookbook/development_tools/debugging/config_log.html
5. 上游优化和联调: seemless不工作的的问题，也许后续会修?https://www.reddit.com/r/rustdesk/comments/1cuum90/i_cant_seem_to_make_rustdesk_service_work_on_linux/
6. 支持 SD： oobabooga/text-generation-webui
   Project page: https://github.com/oobabooga/text-generation-webui
7. 支持Gnome桌面环境联调: 使用elogind，以支持在容器里面(非systemd环境)运行gnome
    icewm还是太简单，gnome最强大和完善
    xfce(任务栏无法显示)=>icewm(太简单)=>gnome(最强大)
    技术参考: https://chatgpt.com/share/a7f1c5c9-1907-4580-b84e-d5d1d8717d51
8. Rootless docker
   Colab基于 ubuntu 镜像, 不支持elogind，所以需要内嵌docker来执行?
8. 优化输入法: fcitx vs fcitx5 vs ibus
9. 优化DPI显示优化: 提高DPI或者降低分辨率

# [写在前面] 记事本保活

避免被系统中断和清理

In [ ]:
# @title 1. 记事本保活 { display-mode: "form" }

# @markdown 第一步: 播放音乐，使得页面看起来永远活动
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

# [主要步骤] 软件和环境准备

TBD: 为什么需要dbus-x11/dbus-lanunch之类的软件和技术，因为现代软件普遍依赖dbus机制来通讯，比如说ibus的设置和协调；必要时可以通过从命令行运行程序来检查报错，一半可以看到dbus相关错误

## 1. 安装操作系统部件

In [ ]:
%%bash
{
nproc
yes|unminimize
apt update
apt install sudo htop aptitude jq apt-file vim xzip screen x11-apps \
xserver-xorg-video-dummy  aptitude sudo mc xterm fuse xdotool \
python3-xdo libxdo-dev xdotool x11-utils mesa-utils btop icewm mc dbus-x11 uuid fonts-noto fonts-noto-cjk \
bash-completion pulseaudio-utils x11-xserver-utils pavucontrol gnome-terminal nautilus-extension-gnome-terminal -y
} &> /dev/null

## 2. 安装chrome


In [ ]:
%%bash
bash <<'EOF'
{
  wget -c https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
   sudo dpkg -i google-chrome-stable_current_amd64.deb
   sudo apt-get install -f
} &> /dev/null
EOF

## 3. 安装LM Studio & Rustdesk & Others

In [ ]:
%%bash
{
  rm rustdesk*.deb || :
  latest_rustdesk=$(curl -s https://api.github.com/repos/rustdesk/rustdesk/releases/latest | jq -r ".tag_name")
  # wget -c $(curl -s https://api.github.com/repos/rustdesk/rustdesk/releases | grep -oP 'https://github.com/rustdesk/rustdesk/releases/download/nightly/rustdesk.*x86_64.deb') &> /dev/null \
  #   echo "rust desk download failed!"

  wget -c https://github.com/rustdesk/rustdesk/releases/download/${latest_rustdesk}/rustdesk-${latest_rustdesk}-x86_64.deb

  latest_jan=$(curl -s https://api.github.com/repos/janhq/jan/releases/latest | jq -r ".tag_name")
  wget -c https://github.com/janhq/jan/releases/download/v${latest_jan}/jan-linux-x86_64-${latest_jan}.AppImage  \
    &> /dev/null || echo "janhq download failed!"

  wget -c https://releases.lmstudio.ai/linux/x86/0.2.26/beta/LM_Studio-0.2.26.AppImage \
    &> /dev/null || echo "lmstudio download failed!"

  dpkg -i rustdesk-*.deb
  apt --fix-broken install -y

  chmod +x *.AppImage
} &> /dev/null

## 4. 中文支持(需要？)

In [ ]:
%%bash
bash <<'EOF'
{
    sudo apt install ibus ibus-pinyin language-pack-zh-hans -y
    # Add environment variables to ~/.bashrc or ~/.xprofile
    echo 'export GTK_IM_MODULE=ibus' >> ~/.bashrc
    echo 'export QT_IM_MODULE=ibus' >> ~/.bashrc
    echo 'export XMODIFIERS=@im=ibus' >> ~/.bashrc
    echo 'ibus-daemon -d -x' >> ~/.bashrc
} &> /dev/null
EOF

### 设置输入法(只需一次)

In [ ]:
%%bash
#ibus-setup  # 手动配置输入法,要在UI里面
#ibus restart

## 5. 启用cloudflare通道 tunnel(可选)

In [ ]:
%%bash
echo "Use your own key and commit 'exit 0'"
exit 0
your_key=xxx
{
    curl -L --output cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb &&

    sudo dpkg -i cloudflared.deb &&

    sudo cloudflared service install $your_key
} &> /dev/null


## 6. 准备X Window相关资源和脚本

**Tips:** 
1. Use **cvt your_width your_height** to get modeline response to your screen
2. Add new modeline for your screen
3. Change Modes to your screen modeline name

In [ ]:
%%writefile xorg-wip.conf

# This xorg configuration file is meant to be used by xpra
# to start a dummy X11 server.
# For details, please see:
# https://xpra.org/Xdummy.html

Section "ServerFlags"
  Option "DontVTSwitch" "true"
  Option "AllowMouseOpenFail" "true"
  Option "PciForceNone" "true"
  Option "AutoEnableDevices" "false"
  Option "AutoAddDevices" "false"
EndSection

Section "InputDevice"
  Identifier "dummy_mouse"
  Option "CorePointer" "true"
  Driver "void"
EndSection

Section "InputDevice"
  Identifier "dummy_keyboard"
  Option "CoreKeyboard" "true"
  Driver "void"
EndSection

Section "Device"
  Identifier "dummy_videocard"
  Driver "dummy"
  #Option "ConstantDPI" "true"
  #VideoRam 4096000
  #VideoRam 256000
  VideoRam 192000
EndSection

Section "Monitor"
  Identifier "dummy_monitor"
  HorizSync   5.0 - 1000.0
  VertRefresh 5.0 - 200.0
  Modeline "1440x810"   95.00  1440 1520 1664 1888  810 813 818 841 -hsync +vsync
  Modeline "2880x1620"  396.25  2880 3096 3408 3936  1620 1623 1628 1679 -hsync +vsync
EndSection

Section "Screen"
  Identifier "dummy_screen"
  Device "dummy_videocard"
  Monitor "dummy_monitor"
  DefaultDepth 24
  SubSection "Display"
    Viewport 0 0
    Depth 24
    #Modes "2880x1620"
    #Tips: 这是分辨率调小一半(类似DPI增大一倍?)
    Modes "1440x810"
  EndSubSection
EndSection

Section "ServerLayout"
  Identifier   "dummy_layout"
  Screen       "dummy_screen"
  InputDevice  "dummy_mouse"
  InputDevice  "dummy_keyboard"
EndSection

#Section "Module"
#    Load "glamoregl" #?
#EndSection

In [ ]:
%%writefile startserver.sh
#!/bin/bash

#/usr/lib/rustdesk/rustdesk &
#xvfb-run xrandr

tee hi_rustdesk.sh <<-EOF
  #!/bin/bash
  #export XDG_SESSION_ID=xxxxxx # 通过错误，使得rustdesk的bug跳过？(新版已经修复)
  #export XDG_SESSION_TYPE=x11 # 没用？
  #export DISPLAY=:0
  #startxfce4 &
  #xfwm4 -c off -r  -b off &
  eval `dbus-launch --sh-syntax` # ？？
  #pulseaudio --daemonize=N --realtime -k
  #pulseaudio --daemonize=N --realtime
  pulseaudio --daemonize=N --realtime &> pulse_audio.log &
  #pulseaudio --daemonize=N --realtime &
  #pulseaudio --daemonize=N --realtime &
  #sleep 5 #? TODO
  pavucontrol&
  icewm &
  #wmaker &
  xterm &
  
  ./LM_Studio-*.AppImage --no-sandbox &
  google-chrome --no-sandbox "https://youtube.com" &
  #sleep 1 #?
  ibus engine pinyin &
  #ibus engine pinyin
  gnome-terminal &
  nautilus &
  xrdb -query | grep dpi
  
  rustdesk &
EOF

chmod +x hi_rustdesk.sh

#killall rustdesk
killall  -9 dbus-launch
killall  -9 Xorg
killall  -9 rustdesk
#killall Xvfb
killall  -9 Xvfb
#killall xfdesktop
killall  -9 xfdesktop
killall  -9 lm-studio
killall  -9 icewm
killall  -9 pulseaudio
killall  -9 gnome-terminal
killall  -9 nautilus

#sleep 1 # TODO?
#xvfb-run /usr/lib/rustdesk/rustdesk

# xvfb-run -s "-screen 0 2560x1440x24" ./hi_rustdesk.sh
#Tips: if some error like futex related, use this one
# export DISPLAY=:44
# xvfb-run --server-num 44 -s "-screen 0 2560x1440x24" ./hi_rustdesk.sh

# /usr/lib/rustdesk/rustdesk --server &

#this does not work
#rustdesk_config_path=~/.config/rustdesk/RustDesk_local.toml
#[ ! -f $rustdesk_config_path ]&&tee $rustdesk_config_path <<EOF
#[options]
#allow-remote-config-modification = Y
#EOF

Xorg -config xorg-wip.conf -noreset +extension GLX +extension RANDR +extension RENDER :1 &
#sleep 1 #?
export DISPLAY=:1

./hi_rustdesk.sh &

#this does not work
#rustdesk_config_path=~/.config/rustdesk/RustDesk2.toml
#grep -q "allow-remote-config-modification = Y" $rustdesk_config_path || echo "allow-remote-config-modification = Y" >> $rustdesk_config_path

#https://github.com/rustdesk/rustdesk/blob/e7f0f0ff8d0478cabe4b9d374025936cbe206584/src/core_main.rs#L356C31-L356C39

tmp_pwd=$(uuid)

until /usr/lib/rustdesk/rustdesk --password $tmp_pwd &>/dev/null ; do
  echo "Rustdesk 启动失败，正在重试..."
  sleep 1
done
echo "id: $(/usr/lib/rustdesk/rustdesk --get-id)";
echo "pwd: $tmp_pwd"

rustdesk --option allow-remote-config-modification Y
rustdesk --option allow-remote-config-modification


In [ ]:
%%bash

#Tips: 目测这些都不生效, 修正后删除 "exit 0"
exit 0

[[ -d ~/.icewm ]] || mkdir ~/.icewm
tee ~/.icewm/preferences << EOF
Xft.dpi: 192
ToolTipFontName="-*-sans-medium-r-*-*-*-120-*-*-*-*-*-*"
StatusFontName="-*-sans-bold-r-*-*-*-120-*-*-*-*-*-*"
MenuFontName="-*-sans-bold-r-*-*-*-120-*-*-*-*-*-*"
ActiveTaskBarFontName="-*-sans-bold-r-*-*-*-120-*-*-*-*-*-*"
NormalTaskBarFontName="-*-sans-medium-r-*-*-*-120-*-*-*-*-*-*"
ListBoxFontName="-*-sans-medium-r-*-*-*-120-*-*-*-*-*-*"
EOF

tee ~/.Xresources <<EOF
Xft.dpi: 192
EOF


## 7. Locate your server if needed

In [ ]:
%%bash
!curl -s https://ipinfo.io | grep '"country"' | awk -F ': ' '{print $2}' | tr -d '",'
!curl -s http://ip-api.com/json | jq -r '.country'

# [主要步骤] 启动&连接

## 1. 启动


In [ ]:
#TBD
!chmod +x ./startserver.sh
import subprocess
#subprocess.run(['./startserver.sh'])
#print(subprocess.run(["./startserver.sh"], capture_output=True).stdout.decode().splitlines())
for line in subprocess.run(["./startserver.sh"], capture_output=True).stdout.decode().splitlines():
  print(line)
!curl -s https://ipinfo.io | grep '"country"' | awk -F ': ' '{print $2}' | tr -d '",'
!curl -s http://ip-api.com/json | jq -r '.country'


## 2. 连接
Connect from Rustdesk and See the magic

# [写在后面] x11docker do all the magic?

后面发现了x11docker项目，感觉有很多值得借鉴的地方，包括技术方案和思路

# [写在后面] 技术调研参考

## 1. 命令行检索rustdesk访问信息

In [ ]:
%%bash
/usr/lib/rustdesk/rustdesk --get-id
tmp_pwd=$(uuid)
/usr/lib/rustdesk/rustdesk --password $tmp_pwd
echo $tmp_pwd
#https://github.com/rustdesk/rustdesk/blob/e7f0f0ff8d0478cabe4b9d374025936cbe206584/src/core_main.rs#L356C31-L356C39
#https://rustdesk.com/docs/en/self-host/client-configuration/advanced-settings/
rustdesk --option allow-remote-config-modification Y
rustdesk --option enable-file-copy-paste Y # just windows works?
rustdesk --option allow-linux-headless Y # need set for server?
#rustdesk --option audio-input = 'Microphone (Display Audio)'
rustdesk --option verification-method = 'use-permanent-password'
rustdesk --option verification-method


## 2. 检查x-window-manager

In [ ]:
%%bash
ls /etc/alternatives/ | grep ^x
#x-session-manager
#x-terminal-emulator
#x-window-manager
sudo update-alternatives --list x-window-manager

## 3. rustdesk headless 实现

https://github.com/rustdesk/rustdesk/pull/3902

https://github.com/rustdesk/rustdesk/pull/3902#issuecomment-1937900476

"--service" vs "--server" # we use --server

## 4. 配置ibus
https://gist.github.com/adnan360/11aed4d206004f32153d83c2d475eb95

In [ ]:
%%bash
#https://gist.github.com/adnan360/11aed4d206004f32153d83c2d475eb9
#ibus engine xkb:us::eng
#ibus engine pinyin


## 5. xvfb-run & wmaker 尝试

In [ ]:
%%bash
exit 0
# ref: https://github.com/rustdesk/rustdesk/wiki/Headless-Linux-Support
# cat /root/.config/rustdesk/*
##xdg-desktop-portal

tee hi_rustdesk.sh <<EOF
export XDG_SESSION_ID=xxxxxx # 通过错误，使得rustdesk的bug跳过？(新版已经修复)
export XDG_SESSION_TYPE=x11 # 没用？
export DISPLAY=:0
EOF

chmod +x hi_rustdesk.sh

# echo . $PWD/hi_rustdesk.sh >>  ~/.bashrc
# echo . $PWD/hi_rustdesk.sh >> /etc/profile

# Xvfb $DISPLAY &

# /usr/lib/rustdesk/rustdesk --service
# /usr/lib/rustdesk/rustdesk --get-id
# /usr/lib/rustdesk/rustdesk --password killcolab

killall wmaker # xface?
killall rustdesk
killall Xvfb

. $PWD/hi_rustdesk.sh
# --begin-- input:
# x=1920
# y=1080
# refresh=60

x=2560
y=1440
refresh=60

# --end-- input:

# Xvfb :1 -screen 0 1024x768x32 &
# Xvfb $DISPLAY -screen 0 1024x768x24
Xvfb $DISPLAY -screen 0 ${x}x${y}x24+8 &

# wmaker 试水
wmaker &
# dbus-run-session gnome-session
/usr/lib/rustdesk/rustdesk &

/usr/lib/rustdesk/rustdesk --get-id

sleep 2
/usr/lib/rustdesk/rustdesk --password killcolab
xwd -root -silent | gm convert xwd:- output.png

# 屏幕调整
# tips: xvfb 屏幕叫screen
screen_name=screen
mode_line=$(cvt $x $y $refresh | grep "Modeline" | cut -d' ' -f2-)
mode_name=$(echo $mode_line | cut -d' ' -f1)
xrandr --newmode $mode_line
xrandr --addmode $screen_name $mode_name
xrandr --output $screen_name --mode $mode_name

# gnome 试水
# 启动 D-Bus session (不工作 TBD)
eval `dbus-launch --sh-syntax --exit-with-session`
eval `ssh-agent -s`
ssh-add
gnome-session

# xdummy 试水(rustdesk 官方配置 TBD)

# lm studio
#./LM_Studio-0.2.20.AppImage  --no-sandbox

# git clone https://github.com/rustdesk/rustdesk.git
# Unsupported display server type "", x11 or wayland expected
# rustdesk/rustdesk/src/server/connection.rs:1181
# rustdesk/rustdesk/libs/hbb_common/src/platform/linux.rs:get_display_server
# rustdesk/rustdesk/libs/hbb_common/src/platform/linux.rs:get_display_server_of_session
# rustdesk/rustdesk/src/platform/linux.rs

### 7. xvfb-run的问题解决

https://github.com/electron/electron/issues/24211

Error:
The futex facility returned an unexpected error code.

Workaround:
```bash
#DISPLAY=:44 xvfb-run --server-num 44
```


## 8. loginctl 验证

In [ ]:
%%bash
exit 0
loginctl show-session -p Type $(cat /proc/self/sessionid)
echo $?

## 9. rustdesk 远程键盘不工作（重启好了）
Ref:
https://github.com/jordansissel/xdotool/issues/156



In [ ]:
%%bash
exit 0
xdotool hello world
xdotool type "hello world"
xdotool getactivewindow type hello

dpkg-reconfigure keyboard-configuration

# 参考

https://xpra.org/xorg.conf

https://ice-wm.org/

https://github.com/Xpra-org/xpra/blob/master/docs/Usage/Xdummy.md

https://github.com/rustdesk/rustdesk/blob/master/res/startwm.sh

https://github.com/rustdesk/rustdesk/pull/3852/files#diff-26550121d7f5c5179c46ff523f3b81787d25f4ab59724e5a0b1b98572b15c87b

https://github.com/rustdesk/rustdesk/blob/master/res/xorg.conf

https://github.com/congzhangzh/daily-scripts-4-us/blob/main/enable_headness_intel_igpu_4_nomachine.sh

https://github.com/rustdesk/rustdesk/blob/17d18f1dd82ba2e02a5cf540fc935e7651c61554/src/platform/linux_desktop_manager.rs#L662

https://github.com/Xpra-org/xf86-video-dummy

https://gitlab.xfce.org/xfce/xfwm4/-/issues/432